In [ ]:
import os
import numpy as np
import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

In [ ]:
# Salida determinística (comentar para aleatorizar el entrenamiento)
np.random.seed(1000)

In [ ]:
# Funciones útiles

def plot_losses(d_losses,g_losses):
    plt.figure(figsize=(10, 8))
    plt.plot(d_losses, label='Discriminitive loss')
    plt.plot(g_losses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Dibujar imagenes generadas
def plot_images(epoch, examples=36, dim=(6, 6), figsize=(5, 5)):
    noise = np.random.normal(0, 1, size=[examples, latent_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.suptitle(f"Epoch {epoch}")
    plt.tight_layout()
    plt.show()

In [ ]:
# Cargar MNIST (solo x_train)

(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32)/255)-0.5
X_train = X_train.reshape(60000, 28,28,1)
output_shape=(28,28,1)

In [ ]:
def GAN(latent_dim,output_shape):
  adam = Adam(learning_rate=0.0002, beta_1=0.5)

  # Red Generadora
  generator = Sequential(name="generator")
  generator.add(Dense(128, input_dim=latent_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(256))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(784, activation='tanh'))
  generator.add(Reshape(output_shape))

  # Red Discriminadora
  discriminator = Sequential(name="discriminator")
  discriminator.add(Flatten(input_shape=output_shape))
  discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer=adam)

  # No entrenar el discriminador al entrenar todo
  discriminator.trainable = False

  # Red GAN completa
  ganInput = Input(shape=(latent_dim,),name="input")
  x = generator(ganInput)
  ganOutput = discriminator(x)
  gan = Model(inputs=ganInput, outputs=ganOutput,name="GAN")
  gan.compile(loss='binary_crossentropy', optimizer=adam)
  return gan,discriminator,generator

latent_dim = 16
gan,discriminator,generator = GAN(latent_dim, output_shape)

print(gan.summary())
print(generator.summary())
print(discriminator.summary())

In [ ]:
# entrenar el GAN

batch_size = 128
epochs = 3

batches = X_train.shape[0] // batch_size
print( f'Epochs:{epochs}')
print( f'Batch size: {batch_size}')
print( f'Batches per epoch: {batches}')

d_losses = []
g_losses = []
for e in range(epochs):
        for _ in range(batches):
            # Generar un vector de valores aleatorios
            noise = np.random.normal(0, 1, size=[batch_size, latent_dim])

            # Generar imágenes falsas de MNIST en base a ese vector
            fake_images_batch = generator.predict(noise)

            # Imagenes reales aleatorias
            real_images_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

            # Genero un batch para el discriminador
            x_discriminator = np.concatenate([real_images_batch, fake_images_batch])

            # Ponemos clase 1 para las imagenes reales  y clase 0 a las imágenes falsas
            y_discriminator = np.concatenate([np.ones(batch_size),np.zeros(batch_size)])

            # Entrenar discriminador
            discriminator.trainable = True

            # train_on_batch es un método del modelo de Keras
            d_loss = discriminator.train_on_batch(x_discriminator, y_discriminator, verbose = False)
            discriminator.trainable = False

            # Entrenar generador (entrena toda la red)
            noise = np.random.normal(0, 1, size=[batch_size, latent_dim])
            y_generator = np.ones(batch_size)

            g_loss = gan.train_on_batch(noise, y_generator)

        # Guardar losses del ultimo batch
        tqdm.tqdm.write( f'Errores: G={g_loss}, D={d_loss}')
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        #dibujar imagenes generadas
        if e == 0 or e % 5 == 0:
            plot_images(e)

plot_losses(d_losses,g_losses)

In [ ]:
# Dibujar imagenes generadas

dim=(4, 4)
figsize=(5, 5)

random_input = np.random.normal(0, 1, size=[dim[0]*dim[1], latent_dim])

generated_images = generator.predict(random_input)

plt.figure(figsize=figsize)
for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
plt.tight_layout()
plt.show()